# **Class Imbalance : Class Weight**



## 0.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

from tqdm import tqdm

# 불필요한 경고 제거
import warnings
warnings.filterwarnings(action='ignore')

### (2) 데이터 로딩

In [ ]:
seed = 28

x, y = make_classification(n_samples=1000,
                        n_features=2,
                        n_redundant=0,
                        weights = [0.95, 0.05],  # class 0과 1의 비율 조정 ==> class imbalance 상황만들기
                        n_clusters_per_class=1,
                        random_state=seed)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
pd.Series(y).value_counts(normalize = True)

### (3) 필요한 함수 만들기


* 데이터 분포 시각화 함수

In [ ]:
def my_scatter(x, y) :

    # y가 넘파이 이므로 시리즈로 변환하고 클래스별 갯수 세기
    temp = pd.Series(y).value_counts()

    plt.figure(figsize=(6, 6))
    plt.title(f'0 : {temp[0]},    1 : {temp[1]}')

    sns.scatterplot(x=x[:,0], y=x[:,1], hue=y)

    plt.xlabel('x1')
    plt.ylabel('x2')
    # plt.xlim(-4, 2.2)
    # plt.ylim(-4, 2.2)
    plt.show()

* svm 모델 시각화
    * 아래 모델은 **feauture 수가 2개일 때만 가능**합니다.
    * 모델 두개를 비교 가능.

In [ ]:
def svm_visualize(x, y, model1, model2 = 0) :

    xx, yy = np.meshgrid(np.linspace(x[:,0].min(), x[:,0].max(), 50),
                         np.linspace(x[:,1].min(), x[:,1].max(), 50)) # mesh grid

    # 메쉬 그리드값에 대해 모델 부터 거리 값 만들기.
    Z = model1.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    if model2 != 0 : # 두번째 모델이 있으면 처리
        Z1 = model2.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z1 = Z1.reshape(xx.shape)

    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=x[:,0], y=x[:,1], hue=y)
    plt.contour(xx, yy, Z, levels=[0], colors = 'gray',linestyles  = '--')

    if model2 != 0 : # 두번째 모델이 있으면 처리
        plt.contour(xx, yy, Z1, levels=[0], colors = 'r',linestyles  = '--')

    plt.axis("tight")
    plt.show()

- 위에서 만든 데이터 분포를 확인합니다.

In [ ]:
my_scatter(x, y)

## 1.Class Weight 조정

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import *

### (1) 기본 모델링

In [ ]:
# 모델링 및 평가
model = SVC(kernel='linear')
model.fit(x, y)
pred = model.predict(x)

print(confusion_matrix(y, pred))
print('=' * 55)
print(classification_report(y, pred))

### (2) class_weight 조정1
* class_weight='balanced'


In [ ]:
# 모델링 및 평가
model1 = SVC(kernel='linear', class_weight='balanced')
model1.fit(x, y)
pred = model1.predict(x)

print(confusion_matrix(y, pred))
print('=' * 55)
print(classification_report(y, pred))

In [ ]:
svm_visualize(x,y, model, model1)

### (3) class_weight 조정2
* class_weight= {0:##, 1:##}


In [ ]:
weight_1 = 0.952
model1 = SVC(kernel='linear', class_weight= { 0:(1-weight_1)  , 1:weight_1} )
model1.fit(x, y)
pred = model1.predict(x)
print(confusion_matrix(y, pred))
print(classification_report(y, pred))
svm_visualize(x,y, model, model1)

## 2.🤔실습 : Semiconductor manufacturing process dataset

![](https://assets.pandaily.com/uploads/2021/10/semiconductor.png)

* 반도체 제조 공정은 시점별로 수많은 센서로부터 정보를 수집하여 공정을 감시합니다.
* 센서정보와 함께 공정간 발생된 불량품에 대한 정보를 저장하였습니다.
* 불량을 예측해 봅시다.

### (1) 데이터 준비

#### 1)Data Loading

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)
data.head()

변수 정보
* label : 1 - 불량, 0 - 정상
* v### : 센서값들


In [ ]:
print(data['label'].value_counts()) # 범주별 개수
print(data['label'].value_counts(normalize = True)) #범주별 비율

#### 2)x, y 로 나누기

In [ ]:
target = 'label'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

#### 3)데이터 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state=10)

In [ ]:
x_train.shape, y_train.shape

### (2) 모델링

In [ ]:
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV

# 필요한 라이브러리, 함수들을 불러옵시다.
from sklearn.linear_model import LogisticRegression

#### 1)Class Weight = 'balanced'

* class weight = 'balanced'로 모델링 및 검증

#### 2)class_weight = 비율로 조절
* 수동 조절